In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
if not os.path.isdir("vdiff_activity_files"):
    os.mkdir("vdiff_activity_files")
from i2x.der_hca import hca, islands as isl, PlotUtils as pltutl
import numpy as np

# Voltage Violations
One of the the violations that shows up fairly regulary with the introduction of VRE is a $\Delta V$ violation. 
According to **ADD REFERENCE** a sudden change of voltage may not exceed 3%.
In thi exercise we'll be exploring a situation where a $\Delta V$ violation occurs and different possibilities to address it.

## Part 1: Setup
First of all, let's set up a problem where this violation shows up.
There are many ways to do this, but in this case we'll follow these steps:
1. let our simulation add resources until we encounter an instance where the capacity is limited due to a $\Delta V$ violation (note: this should happen on round 8).
2. Re-wind that last step and add the resource _allowing_ for violations

In [ ]:
feeder = hca.HCA("./configs/vdiff_activity.json")
feeder.runbase()
while True:
    feeder.hca_round("pv")
    ## get the hosting capacity for the last bus considered, if zero, then likely capacity was limited
    hc, cnt = feeder.get_hc("pv", feeder.visited_buses[-1], feeder.cnt)
    if (hc["kw"] == 0) and ("voltage_vdiff" in feeder.metrics.last_violation_list):
        ## there are violations and they are of the kind we are interested in
        break
    else:
        ## save for later re-wind
        feeder.save("./vdiff_activity_files/vdiff_activity_setup.pkl")

We saved the state of the system following each addition _except_ the one that caused the violation we are interested in.
To re-wind, we simply reload the saved file `pkl` file.

> **Technical Sidenotes** <br>
> * [pickle](https://docs.python.org/3/library/pickle.html) is just a binary format to save python data.
> * To reload we need to pass the `reload=True` flag to the object initialization.
> * It's also useful to pass `reload_filemode="a"`, which just means that the log we're saving to will be appended rather than overwritten.

In [ ]:
feeder = hca.HCA("./vdiff_activity_files/vdiff_activity_setup.pkl", reload=True, reload_filemode="a", logger_heading="\n*****Re-loading pre-vdiff limitation******\n")
feeder.hca_round("pv", allow_violations=True) #re-run allowing for violations
feeder.save("./vdiff_activity_files/vdiff_activity_setup_final.pkl") #save again for later intialization

## Part 2: Analyze
Before we start changing too many things, let's just see where we are right now.

In [ ]:
pltutl.vdiff_plot(feeder, "./vdiff_activity_files/vdiff_activity_part2", 
                  include_plotlyjs='cdn', auto_open=True)

Looking at the voltage difference plots we see that 4 buses are violating the metric: `bus_1110`, `bus_1111`, `bus_1112` (where the unit is located) and `bus_1113`.

![](./figs/vdiff_violation.png) ![](./figs/vdiff_violation_feeder.png)

It is clear from the figures that the sudden changes in output from the new unit `pv_bus_1112_cnt8`, due to the cloudy day profile we're using is responsible for a local, sudden drops in voltage.

## Part 3: Possible Remedies
Now that we've identified and isolated the problem, we can begin to investigate solutions.

> **Recall**
> The purpose of this exercise is to show HCA and Screening tools can be used to study the interconnection process.
> The list of possible solutions here is not intended to be complete.

### Conductor upgrades
The logic behind conductor upgrades is that they lead to a tighter electrical connection between location with issues and the stronger feeder source (substation).
We first look for the longest path between the source bus and the buses where the $Delta V$ requirement is violated.
We then begin to work up this path (towards the source), upgrading the conductor.
After each upgrade we perform two kinds of tests:
1. Are the violations gone?
2. Is there any hosting capacity?


In [ ]:
#########################################################################
####### these are some functions to get the path and perform the upgrades
#########################################################################
def get_upgrade_path(feeder:hca.HCA):
    """get the path from the furthest location from the source bus where
    a voltage difference problem occurs, to the source bus
    """
    ### collect locations where voltage difference is violated
    vdiff_buses = list(feeder.metrics.get_vdiff_locations()["v"].keys())
    sources = []
    upgrade_paths = []
    path_lengths = []
    ### for each location with a violation, find the bath to the source bus
    for b in vdiff_buses:
        nearest_source, path2source = isl.get_nearest_source(feeder.G.to_undirected(), b)
        sources.append(nearest_source)
        upgrade_paths.append(path2source)
        path_lengths.append(len(path2source))
    ### select the maximum distance to the source bus.
    idx = np.argmax(path_lengths)
    return upgrade_paths[idx]

def upgrade_until_no_violation(feeder:hca.HCA, upgrade_path:list):
    """Iterate over the path to the source and upgrade the conductor 
    until no violations
    """
    for u,v in zip(upgrade_path[1:], upgrade_path[:-1]):
        eclass = feeder.G.edges[u,v]["eclass"]
        ename = feeder.G.edges[u,v]["ename"]
        if eclass.lower() == "line":
            feeder.upgrade_line(ename)
        
        ## run dss
        feeder.reset_dss(clear_changes=False)
        feeder.rundss()
        if not feeder.lastres["converged"]:
            raise ValueError("Open DSS did not converge")
        
        feeder.metrics.load_res(feeder.lastres)
        feeder.metrics.calc_metrics()
        
        ## check if there are any violations
        if feeder.metrics.violation_count == 0:
            break

def upgrade_until_some_hc(feeder:hca.HCA, upgrade_path:list):
    """Iterate over the path to the source and upgrade the conductor 
    until the hosting capacity at the last visited bus is non-zero
    (implies also no violations)
    """
    for u,v in zip(upgrade_path[1:], upgrade_path[:-1]):
        eclass = feeder.G.edges[u,v]["eclass"]
        ename = feeder.G.edges[u,v]["ename"]
        if eclass.lower() == "line":
            feeder.upgrade_line(ename)
        
        ## run dss
        feeder.reset_dss(clear_changes=False)
        feeder.rundss()
        if not feeder.lastres["converged"]:
            raise ValueError("Open DSS did not converge")
        
        feeder.hca_round("pv", bus=feeder.visited_buses[-1], recalculate=True)
        hc, cnt = feeder.get_hc("pv", feeder.visited_buses[-1], feeder.cnt)
        
        ## check if hosting capacity is non-zero
        if hc["kw"] > 0:
            break

#### Step 1: Get the path towards the source

In [ ]:
feeder = hca.HCA("./vdiff_activity_files/vdiff_activity_setup_final.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading="\n*****Re-loading post-vdiff violation******\n")
feeder.metrics.load_res(feeder.lastres)
feeder.metrics.calc_metrics()
upgrade_path = get_upgrade_path(feeder)
feeder.logger.info(f"Furthest violation from substation on bus {upgrade_path[0]}")

#### Step 2a: Perform upgrades until no mo violations

In [ ]:
feeder = hca.HCA("./vdiff_activity_files/vdiff_activity_setup_final.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading="\n*****Re-loading post-vdiff violation******\n")
upgrade_until_no_violation(feeder, upgrade_path)
feeder.logger.info(f"Vdiff metric post upgrades: {feeder.metrics.vdiff}")
feeder.logger.info(f"Estimated upgrade cost:")
feeder.logger.info(f"\tLines: ${sum(v[feeder.cnt]['cost'] for v in feeder.data['upgrades']['line'].values()):0.2f} | {sum(v[feeder.cnt]['length']*hca.conductor_cost.units2ft(v[feeder.cnt]['length_unit']) for v in feeder.data['upgrades']['line'].values()):0.2f} ft")
feeder.hca_round("pv", bus=feeder.visited_buses[-1], recalculate=True)
pltutl.upgrade_plot(feeder, "./vdiff_activity_files/vdiff_activity_conductor_upgrade1", include_plotlyjs='cdn', auto_open=True)
feeder.save("./vdiff_activity_files/vdiff_activity_conductor_upgrade1.pkl")

While this technically worked, there is no capacity on the feeder.
Locations around `bus_1112` will similarly not be able to accommodate anything further.
> _Extra Exercise_:
> Test this statement out! Starting with the results in this state and try to add a resource at a nearby location and see whether any capacity is available.

An alternative approach could be to upgrade until the capacity is non-zero.
#### Step 2b: Perform upgrades until non-zero hosting capacity

In [ ]:
## reload state just to make sure we're not double-counting anywhere
feeder = hca.HCA("./vdiff_activity_files/vdiff_activity_setup_final.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading="\n*****Re-loading post-vdiff violation******\n")
upgrade_until_some_hc(feeder, upgrade_path)
feeder.logger.info(f"Vdiff metric post upgrades: {feeder.metrics.vdiff}")
feeder.logger.info(f"Estimated upgrade cost:")
feeder.logger.info(f"\tLines: ${sum(v[feeder.cnt]['cost'] for v in feeder.data['upgrades']['line'].values()):0.2f} | {sum(v[feeder.cnt]['length']*hca.conductor_cost.units2ft(v[feeder.cnt]['length_unit']) for v in feeder.data['upgrades']['line'].values()):0.2f} ft")
pltutl.upgrade_plot(feeder, "./vdiff_activity_files/vdiff_activity_conductor_upgrade2", include_plotlyjs='cdn', auto_open=True)
feeder.save("./vdiff_activity_files/vdiff_activity_conductor_upgrade2.pkl")

### Capacitor
Capacitors are frequently used to deal with both reactive power as well as voltage issues.
So we try to see whether a capacitor may help.
In the cell below we load the change the `bus` and `kvar` values and re-run the cell.
The output will show the $\Delta V$ pre- and post-capacitor additions.
After a bit of playing around it should become rather evident that a capacitor is unlikely to help in this situation.

In [ ]:
bus = "bus_1102"
kvar= 1200
feeder = hca.HCA("./vdiff_activity_files/vdiff_activity_setup_final.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading="\n*****Re-loading post-vdiff violation******\n")
feeder.metrics.load_res(feeder.lastres)
feeder.metrics.calc_metrics()
feeder.logger.info(f"Vdiff pre-Capacitor: {feeder.metrics.vdiff:0.2f}%")
feeder.logger.info(f"Adding {kvar} kVAr capacitor at bus {bus}")
feeder.dss.text(f"new capacitor.cap_{bus} bus1={bus} phases=3 kvar={kvar} kv=12.47 con=wye")
feeder.rundss()
feeder.metrics.load_res(feeder.lastres)
feeder.metrics.calc_metrics()
feeder.logger.info(f"Vdiff post-Capacitor: {feeder.metrics.vdiff:0.2f}%")

### Inverter Controls
An alternative solution is to activate advanced inverter controls [**CITE 1547**] that enable more control of the POI voltage via reactive as well as possibly real power adjustments.

The choices available in this program are:
```
'CONSTANT_PF', 'VOLT_WATT', 'VOLT_VAR_CATA', 'VOLT_VAR_CATB', 'VOLT_VAR_AVR', 'VOLT_VAR_VOLT_WATT', 'VOLT_VAR_14H'
```
In the cell below, change the choices to see the different curves.

In [ ]:
inverter_choice="VOLT_VAR_CATB"
pltutl.inverter_control_plot(inverter_choice)

To see the impact we'll re-run the simulation from the beginning but this with different inverter controls enabled.
The exercise begins with `VOLT_VAR_CATB` but feel free to experiment with the different options and see what impact it has.

First we'll run exactly the same scenario as before, i.e 8 additions of exactly the same size:

In [ ]:
inverter_choice="VOLT_VAR_CATB"
feeder = hca.HCA("./configs/vdiff_activity.json", logger_heading=f"\n****** Using Inverter Mode {inverter_choice} ********\n")
feeder.logger.info(f"Changing inverter control from {feeder.inputs['invmode']} to {inverter_choice}")
feeder.inputs["invmode"] = inverter_choice
feeder.runbase()
for i in range(8):
    # on the last round allow violations so we have a comparable scenario to before
    feeder.hca_round("pv", allow_violations=i==7)
pltutl.vdiff_plot(feeder, f"./vdiff_activity_files/vdiff_activity_{inverter_choice}", include_plotlyjs='cdn', auto_open=True)
feeder.save(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}.pkl")

### Things to think about
* Take a look at the $\Delta V$ curves. Is the violation as severe as before, or a bit tampered?
* How about the number of buses where a $\Delta V$ violation is recorded?

#### Add Conductor upgrade
The inverter controls in this case are not enough to avoid any upgrades.
To test what happens when we upgrade, we try the same strategies from [before](#conductor-upgrades):
1. Upgrade until no violation
2. Upgrade until some hc

In [ ]:
feeder = hca.HCA(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading=f"\n*****Re-loading post-vdiff violation {inverter_choice}******\n")
feeder.metrics.load_res(feeder.lastres)
feeder.metrics.calc_metrics()
upgrade_path = get_upgrade_path(feeder)
feeder.logger.info(f"Furthest violation from substation on bus {upgrade_path[0]}")

In [ ]:
feeder = hca.HCA(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading=f"\n*****Re-loading post-vdiff violation {inverter_choice}******\n")
upgrade_until_no_violation(feeder, upgrade_path)
feeder.logger.info(f"Vdiff metric post upgrades: {feeder.metrics.vdiff}")
feeder.logger.info(f"Estimated upgrade cost:")
feeder.logger.info(f"\tLines: ${sum(v[feeder.cnt]['cost'] for v in feeder.data['upgrades']['line'].values()):0.2f} | {sum(v[feeder.cnt]['length']*hca.conductor_cost.units2ft(v[feeder.cnt]['length_unit']) for v in feeder.data['upgrades']['line'].values()):0.2f} ft")
feeder.hca_round("pv", bus=feeder.visited_buses[-1], recalculate=True)
pltutl.upgrade_plot(feeder, f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade1", include_plotlyjs='cdn', auto_open=True)
feeder.save(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade1.pkl")

In [ ]:
## reload state just to make sure we're not double-counting anywhere
feeder = hca.HCA(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}.pkl", 
                 reload=True, reload_filemode="a", 
                 logger_heading=f"\n*****Re-loading post-vdiff violation {inverter_choice}******\n")
upgrade_until_some_hc(feeder, upgrade_path)
feeder.logger.info(f"Vdiff metric post upgrades: {feeder.metrics.vdiff}")
feeder.logger.info(f"Estimated upgrade cost:")
feeder.logger.info(f"\tLines: ${sum(v[feeder.cnt]['cost'] for v in feeder.data["upgrades"]['line'].values()):0.2f} | {sum(v[feeder.cnt]['length']*hca.conductor_cost.units2ft(v[feeder.cnt]['length_unit']) for v in feeder.data["upgrades"]['line'].values()):0.2f} ft")
pltutl.upgrade_plot(feeder, f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade2", include_plotlyjs='cdn', auto_open=True)
feeder.save(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade2.pkl")

## Part 4: Comparison Between Solutions


In [ ]:
def compare_cases(feeder_a, feeder_b):
    """Compare a constant PF scenario (feeder_a)
    To an inverter function scenario (feeder_b)
    """
    ## HC comparison
    print(f"HC improvement when inverter control is set to {inverter_choice}:")
    display((feeder_b.get_data("hc", "pv") - feeder_a.get_data("hc", "pv")).sum())

    ## Cost comparison
    bus = feeder_b.visited_buses[-1]
    Sij, cnt = feeder_b.get_data("Sij", "pv", bus)
    cost_a = sum(v[feeder_a.cnt]["cost"] for v in feeder_a.data.['upgrades']["line"].values())
    cost_b = sum(v[feeder_b.cnt]["cost"] for v in feeder_b.data.['upgrades']["line"].values())
    print(f"Upgrade cost savings at bus {bus} to add {Sij['kw']} kW/{Sij['kva']} kVA capacity:")
    print(f"With out {inverter_choice}: ${cost_a:0.2f}")
    print(f"with {inverter_choice}: ${cost_b:0.2f}")
    print(f"Savings: ${cost_a-cost_b:0.2f}")

### Where conductors were upgraded just to avoid violation

In [ ]:
## Load the two feeder
feeder_a = hca.HCA("./vdiff_activity_files/vdiff_activity_conductor_upgrade1.pkl", reload=True, reload_filemode="a")
feeder_b = hca.HCA(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade1.pkl", reload=True, reload_filemode="a")
compare_cases(feeder_a, feeder_b)

### When concuctors were upgraded until non-zero HC was reached

In [ ]:
## Load the two feeder
feeder_a = hca.HCA("./vdiff_activity_files/vdiff_activity_conductor_upgrade2.pkl", reload=True, reload_filemode="a")
feeder_b = hca.HCA(f"./vdiff_activity_files/vdiff_activity_{inverter_choice}_conductor_upgrade2.pkl", reload=True, reload_filemode="a")
compare_cases(feeder_a, feeder_b)

## Summary Take-Aways
1. As more VRE, resources are added to the system, the maximum $\Delta V$ of 3% can become binding.
2. Possible solutions investigated:
    a. Upgrade path towards source to improve system strength and thus reduce voltage volatility
    b. Insert capacitor (not very effective)
    c. Utilize advance inverter functions
3. While advanced inverter functions did not resolve the $\Delta V$ issue on their own, they:
    * Increase the hosting capacity as a whole (~475 kW in the example shown)
    * Reduce the necessary upgrades (and thus cost) necessary to alleviate violations (~250k saving). 